In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm import tqdm
import numpy as np
import pandas as pd 
import CococoNet_reader
import Name_resolver
import os
import itertools

In [2]:
mapped_orthogroups = pd.read_csv('/data/passala/Collaborator_Data/Nick_cross_species_evolution/Updated_orthogroups/all_mappe_nicholas_orthogroups.csv')
clean_orthogroups = pd.read_csv('/data/passala/Collaborator_Data/Nick_cross_species_evolution/Updated_orthogroups/tidy_formatted_orthogroups.csv')

In [3]:
clean_orthogroups_with_ncbi = clean_orthogroups.merge(mapped_orthogroups[['Protein','Real NCBI Gene']], on = 'Protein', how = 'left')
clean_orthogroups_with_ncbi

,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,Brapa,LOC103848115
1,OG0000000,XP_009123420.1,Brapa,LOC103848216
2,OG0000000,XP_009123559.2,Brapa,LOC103848395
3,OG0000000,XP_009124167.1,Brapa,LOC103849105
4,OG0000000,XP_009135775.2,Brapa,LOC103859933
...,...,...,...,...
204629,OG0025535,XP_048637327.1,Bnapus,NaN
204630,OG0025536,XP_048637463.1,Bnapus,LOC125609897
204631,OG0025536,XP_048637464.1,Bnapus,LOC125609898
204632,OG0025537,XP_048637850.1,Bnapus,LOC106347848


In [4]:
clean_orthogroups_with_ncbi.loc[clean_orthogroups_with_ncbi['Species'] == 'Athaliana', 'Real NCBI Gene'] = clean_orthogroups_with_ncbi.loc[clean_orthogroups_with_ncbi['Species'] == 'Athaliana', 'Protein']
clean_orthogroups_with_ncbi


,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,Brapa,LOC103848115
1,OG0000000,XP_009123420.1,Brapa,LOC103848216
2,OG0000000,XP_009123559.2,Brapa,LOC103848395
3,OG0000000,XP_009124167.1,Brapa,LOC103849105
4,OG0000000,XP_009135775.2,Brapa,LOC103859933
...,...,...,...,...
204629,OG0025535,XP_048637327.1,Bnapus,NaN
204630,OG0025536,XP_048637463.1,Bnapus,LOC125609897
204631,OG0025536,XP_048637464.1,Bnapus,LOC125609898
204632,OG0025537,XP_048637850.1,Bnapus,LOC106347848


In [5]:
clean_orthogroups_with_ncbi = clean_orthogroups_with_ncbi.dropna(subset = ['Real NCBI Gene'])
clean_orthogroups_with_ncbi

,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,Brapa,LOC103848115
1,OG0000000,XP_009123420.1,Brapa,LOC103848216
2,OG0000000,XP_009123559.2,Brapa,LOC103848395
3,OG0000000,XP_009124167.1,Brapa,LOC103849105
4,OG0000000,XP_009135775.2,Brapa,LOC103859933
...,...,...,...,...
204626,OG0025534,XP_048637085.1,Bnapus,LOC125609605
204627,OG0025534,XP_048637087.1,Bnapus,LOC125609607
204630,OG0025536,XP_048637463.1,Bnapus,LOC125609897
204631,OG0025536,XP_048637464.1,Bnapus,LOC125609898


In [6]:
clean_orthogroups_with_ncbi = clean_orthogroups_with_ncbi.replace(to_replace={'Bnapus':'rapeseed','Athaliana':'arabidopsis','Brapa':'mustard'})
clean_orthogroups_with_ncbi

,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,mustard,LOC103848115
1,OG0000000,XP_009123420.1,mustard,LOC103848216
2,OG0000000,XP_009123559.2,mustard,LOC103848395
3,OG0000000,XP_009124167.1,mustard,LOC103849105
4,OG0000000,XP_009135775.2,mustard,LOC103859933
...,...,...,...,...
204626,OG0025534,XP_048637085.1,rapeseed,LOC125609605
204627,OG0025534,XP_048637087.1,rapeseed,LOC125609607
204630,OG0025536,XP_048637463.1,rapeseed,LOC125609897
204631,OG0025536,XP_048637464.1,rapeseed,LOC125609898


In [7]:
clean_orthogroups_with_ncbi.value_counts('Species')

Species
rapeseed       86451
mustard        40564
arabidopsis    26663
dtype: int64

In [8]:
clean_orthogroups_with_ncbi = clean_orthogroups_with_ncbi.drop_duplicates(subset = 'Real NCBI Gene')

In [9]:
just_brapa_and_athaliana = clean_orthogroups_with_ncbi.loc[(clean_orthogroups_with_ncbi['Species'] == 'arabidopsis') | (clean_orthogroups_with_ncbi['Species'] == 'mustard')]
just_brapa_and_athaliana

,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,mustard,LOC103848115
1,OG0000000,XP_009123420.1,mustard,LOC103848216
2,OG0000000,XP_009123559.2,mustard,LOC103848395
3,OG0000000,XP_009124167.1,mustard,LOC103849105
4,OG0000000,XP_009135775.2,mustard,LOC103859933
...,...,...,...,...
204014,OG0025193,XP_033148476.1,mustard,LOC103868320
204016,OG0025194,XP_033148550.1,mustard,LOC103874885
204018,OG0025195,XP_033148563.1,mustard,LOC117134375
204020,OG0025196,XP_033148644.1,mustard,LOC103869108


In [10]:
just_bnapus_and_athaliana = clean_orthogroups_with_ncbi.loc[(clean_orthogroups_with_ncbi['Species'] == 'arabidopsis') | (clean_orthogroups_with_ncbi['Species'] == 'rapeseed')]
just_bnapus_and_athaliana

,Orthogroup,Protein,Species,Real NCBI Gene
85,OG0000000,XP_013645442.1,rapeseed,LOC106350046
86,OG0000000,XP_013650512.1,rapeseed,LOC106355038
87,OG0000000,XP_013650533.1,rapeseed,LOC106355064
88,OG0000000,XP_013650874.2,rapeseed,LOC106355475
89,OG0000000,XP_013651348.2,rapeseed,LOC106356051
...,...,...,...,...
204626,OG0025534,XP_048637085.1,rapeseed,LOC125609605
204627,OG0025534,XP_048637087.1,rapeseed,LOC125609607
204630,OG0025536,XP_048637463.1,rapeseed,LOC125609897
204631,OG0025536,XP_048637464.1,rapeseed,LOC125609898


In [11]:
just_brapa_and_athaliana[75:100]

,Orthogroup,Protein,Species,Real NCBI Gene
84,OG0000000,XP_033148585.1,mustard,LOC117134389
437,OG0000001,AT1G53705,arabidopsis,AT1G53705
438,OG0000001,AT1G67020,arabidopsis,AT1G67020
439,OG0000001,AT3G29750,arabidopsis,AT3G29750
440,OG0000001,AT3G30770,arabidopsis,AT3G30770
441,OG0000001,AT4G00155,arabidopsis,AT4G00155
442,OG0000001,AT4G13320,arabidopsis,AT4G13320
443,OG0000001,AT5G25920,arabidopsis,AT5G25920
444,OG0000001,AT5G27800,arabidopsis,AT5G27800
445,OG0000001,ATMG00860,arabidopsis,ATMG00860


In [12]:
just_bnapus_and_brapa = clean_orthogroups_with_ncbi.loc[(clean_orthogroups_with_ncbi['Species'] == 'rapeseed') | (clean_orthogroups_with_ncbi['Species'] == 'mustard')]
just_bnapus_and_brapa

,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,mustard,LOC103848115
1,OG0000000,XP_009123420.1,mustard,LOC103848216
2,OG0000000,XP_009123559.2,mustard,LOC103848395
3,OG0000000,XP_009124167.1,mustard,LOC103849105
4,OG0000000,XP_009135775.2,mustard,LOC103859933
...,...,...,...,...
204626,OG0025534,XP_048637085.1,rapeseed,LOC125609605
204627,OG0025534,XP_048637087.1,rapeseed,LOC125609607
204630,OG0025536,XP_048637463.1,rapeseed,LOC125609897
204631,OG0025536,XP_048637464.1,rapeseed,LOC125609898


In [14]:
def Calculate_Score_list_for_thresholding_and_use_alternative_list_for_ogs(Species_1 = 'rice',Species_2 = 'maize',outside_orthogroups = pd.DataFrame()):
    import pandas as pd
    import CococoNet_reader
    #Get Species Names in Common form 
    common_name_1 = Name_resolver.species_name_resolver(Species_1,'common')
    common_name_2 = Name_resolver.species_name_resolver(Species_2,'common')
    if common_name_1 == 'rice':
        common_name_1 = 'rice_jp'
    if common_name_2 == 'rice':
        common_name_2 ='rice_jp'
    file_list = os.listdir('/data/passala/OrthoDB_data/V_11_pairwise_maps_fixed_problem_species')
    spec_1_filter = [i for i in file_list if common_name_1 in i]
    spec_2_filter = [i for i in spec_1_filter if common_name_2 in i]
    current_location = '/data/passala/OrthoDB_data/V_11_pairwise_maps_fixed_problem_species/' + spec_2_filter[0] #get string for map

    cross_species_n_m_genes = pd.read_csv(current_location)
    orig_column_common_name_1 = common_name_1 + ' Symbol'
    orig_column_common_name_2 = common_name_2 + ' Symbol'
    cross_species_n_m_genes = cross_species_n_m_genes.rename(columns = {orig_column_common_name_1:common_name_1,orig_column_common_name_2:common_name_2})
    ### Get one to ones
    cross_species_map_one_to_one = cross_species_n_m_genes.drop_duplicates(subset=common_name_1, keep= False,)
    cross_species_map_one_to_one = cross_species_map_one_to_one.drop_duplicates(subset= common_name_2, keep= False)

    ## Convert to Dictionary
    dictionary_mapper_one_to_two = cross_species_map_one_to_one.set_index(common_name_1).to_dict()[common_name_2]
    dictionary_mapper_dos_to_uno = cross_species_map_one_to_one.set_index(common_name_2).to_dict()[common_name_1]

    ## Read In Cococonets 
    coconet_species_one = CococoNet_reader.read_cococonet(common_name_1)
    coconet_species_two = CococoNet_reader.read_cococonet(common_name_2)

    cross_species_n_m_genes['Group ID'] = 'Unassigned'


 


    #Identify Pairs for evaluation
    all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [common_name_1,common_name_2,'Group Number'])
    for group_number in tqdm(list(set(outside_orthogroups['Orthogroup'].to_list()))):
        current_gene_map = outside_orthogroups.loc[outside_orthogroups['Orthogroup'] == group_number]
        list_of_species_1_genes_in_group = list(set(current_gene_map.loc[current_gene_map['Species'] == common_name_1, 'Real NCBI Gene'].to_list()))
        list_of_species_2_genes_in_group = list(set(current_gene_map.loc[current_gene_map['Species'] == common_name_2, 'Real NCBI Gene'].to_list()))
        all_combo_list_current_genes = itertools.product(list_of_species_1_genes_in_group,list_of_species_2_genes_in_group)
        all_combo_list_current_genes = list(map(list,all_combo_list_current_genes))
        current_list_of_pairs = pd.DataFrame(all_combo_list_current_genes,columns = [common_name_1,common_name_2])
        current_list_of_pairs['Group Number'] = group_number
        all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.append(current_list_of_pairs)



    all_pairs_to_evaluate_for_functional_conservation['Species 1 Score'] = np.nan
    all_pairs_to_evaluate_for_functional_conservation['Species 2 Score'] = np.nan

    ## Trim cococonets to match


    trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(cross_species_n_m_genes[common_name_1].to_list())]
    trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(cross_species_n_m_genes[common_name_1].to_list())]
    double_species_1_trimmed_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.columns.intersection(cross_species_map_one_to_one[common_name_1].to_list())]
    double_species_1_trimmed_cococonet = double_species_1_trimmed_cococonet.replace(1,0)

    trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(cross_species_n_m_genes[common_name_2].to_list())]
    trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(cross_species_n_m_genes[common_name_2].to_list())]
    double_species_2_trimmed_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.columns.intersection(cross_species_map_one_to_one[common_name_2].to_list())]
    double_species_2_trimmed_cococonet = double_species_2_trimmed_cococonet.replace(1,0)

    all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[common_name_1].isin(double_species_1_trimmed_cococonet.index)]
    all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[common_name_2].isin(double_species_2_trimmed_cococonet.index)]


    ## Rank
    species_1_cococonet_ranked = trimmed_species_1_cococonet.rank()
    species_2_cococonet_ranked = trimmed_species_2_cococonet.rank()

    #Do top 10 Genes
    top_10_species_1_genes = np.array(
        [double_species_1_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_1_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_1_genes_dataframe = pd.DataFrame(
        data=top_10_species_1_genes,
        index=double_species_1_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    #Convert 
    top_10_species_1_genes_as_species_2 = top_10_species_1_genes_dataframe.replace(to_replace=dictionary_mapper_one_to_two)

    # Get genes for checking 
    have_species_1_pairs = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[common_name_1].isin(top_10_species_1_genes_as_species_2.index)]
    trimmed_all_gene_pairs_for_fc = have_species_1_pairs.loc[have_species_1_pairs[common_name_2].isin(trimmed_species_2_cococonet.index)]
    trimmed_all_gene_pairs_for_fc = trimmed_all_gene_pairs_for_fc.reset_index(drop = True)

    # Get values in species 2 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][common_name_1]
        current_species_2_gene = two_genes[1][common_name_2]
        finger_print_genes = top_10_species_1_genes_as_species_2.loc[current_species_1_gene].to_list()
        gene_ranks_in_species_2 = species_2_cococonet_ranked.loc[species_2_cococonet_ranked.index.isin(finger_print_genes), current_species_2_gene]
        avg_rank_in_species_2 = gene_ranks_in_species_2.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.at[index_from_pairs, 'Species 1 Score'] = avg_rank_in_species_2

    #Repeat for Species 2 

    top_10_species_2_genes = np.array(
        [double_species_2_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_2_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_2_genes_dataframe = pd.DataFrame(
        data=top_10_species_2_genes,
        index=double_species_2_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )


    #convert 
    top_10_species_2_genes_as_species_1 = top_10_species_2_genes_dataframe.replace(to_replace=dictionary_mapper_dos_to_uno)


    # Get values in species 1 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][common_name_1]
        current_species_2_gene = two_genes[1][common_name_2]
        finger_print_genes = top_10_species_2_genes_as_species_1.loc[current_species_2_gene].to_list()
        gene_ranks_in_species_1 = species_1_cococonet_ranked.loc[species_1_cococonet_ranked.index.isin(finger_print_genes), current_species_1_gene]
        avg_rank_in_species_1 = gene_ranks_in_species_1.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.loc[index_from_pairs, 'Species 2 Score'] = avg_rank_in_species_1

    #Caluclate Divisors 
    Number_of_species_1_genes = len(top_10_species_1_genes_as_species_2)
    Number_of_species_2_genes = len(top_10_species_2_genes_as_species_1)


    species_1_score_divisor = Number_of_species_2_genes - 4.5
    species_2_score_divisor = Number_of_species_1_genes-4.5

    #Divide and Average 
    trimmed_all_gene_pairs_for_fc['Species 1 Score'] = trimmed_all_gene_pairs_for_fc['Species 1 Score']/species_1_score_divisor
    trimmed_all_gene_pairs_for_fc['Species 2 Score'] = trimmed_all_gene_pairs_for_fc['Species 2 Score']/species_2_score_divisor
    trimmed_all_gene_pairs_for_fc['Total Score'] = trimmed_all_gene_pairs_for_fc[['Species 1 Score','Species 2 Score']].mean(axis = 1)

    return trimmed_all_gene_pairs_for_fc
    

In [15]:
Species_1 = 'mustard'
Species_2 = 'arabidopsis'
outside_orthogroups = just_brapa_and_athaliana

In [16]:
outside_orthogroups

,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,mustard,LOC103848115
1,OG0000000,XP_009123420.1,mustard,LOC103848216
2,OG0000000,XP_009123559.2,mustard,LOC103848395
3,OG0000000,XP_009124167.1,mustard,LOC103849105
4,OG0000000,XP_009135775.2,mustard,LOC103859933
...,...,...,...,...
204014,OG0025193,XP_033148476.1,mustard,LOC103868320
204016,OG0025194,XP_033148550.1,mustard,LOC103874885
204018,OG0025195,XP_033148563.1,mustard,LOC117134375
204020,OG0025196,XP_033148644.1,mustard,LOC103869108


In [17]:
#Get Species Names in Common form 
common_name_1 = Name_resolver.species_name_resolver(Species_1,'common')
common_name_2 = Name_resolver.species_name_resolver(Species_2,'common')
if common_name_1 == 'rice':
    common_name_1 = 'rice_jp'
if common_name_2 == 'rice':
    common_name_2 ='rice_jp'
file_list = os.listdir('/data/passala/OrthoDB_data/V_11_pairwise_maps_fixed_problem_species')
spec_1_filter = [i for i in file_list if common_name_1 in i]
spec_2_filter = [i for i in spec_1_filter if common_name_2 in i]
current_location = '/data/passala/OrthoDB_data/V_11_pairwise_maps_fixed_problem_species/' + spec_2_filter[0] #get string for map

cross_species_n_m_genes = pd.read_csv(current_location)
orig_column_common_name_1 = common_name_1 + ' Symbol'
orig_column_common_name_2 = common_name_2 + ' Symbol'
cross_species_n_m_genes = cross_species_n_m_genes.rename(columns = {orig_column_common_name_1:common_name_1,orig_column_common_name_2:common_name_2})
### Get one to ones
cross_species_map_one_to_one = cross_species_n_m_genes.drop_duplicates(subset=common_name_1, keep= False,)
cross_species_map_one_to_one = cross_species_map_one_to_one.drop_duplicates(subset= common_name_2, keep= False)

## Convert to Dictionary
dictionary_mapper_one_to_two = cross_species_map_one_to_one.set_index(common_name_1).to_dict()[common_name_2]
dictionary_mapper_dos_to_uno = cross_species_map_one_to_one.set_index(common_name_2).to_dict()[common_name_1]

## Read In Cococonets 
coconet_species_one = CococoNet_reader.read_cococonet(common_name_1)
coconet_species_two = CococoNet_reader.read_cococonet(common_name_2)

cross_species_n_m_genes['Group ID'] = 'Unassigned'





#Identify Pairs for evaluation
all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [common_name_1,common_name_2,'Group Number'])
for group_number in tqdm(list(set(outside_orthogroups['Orthogroup'].to_list()))):
    current_gene_map = outside_orthogroups.loc[outside_orthogroups['Orthogroup'] == group_number]
    list_of_species_1_genes_in_group = list(set(current_gene_map.loc[current_gene_map['Species'] == common_name_1, 'Real NCBI Gene'].to_list()))
    list_of_species_2_genes_in_group = list(set(current_gene_map.loc[current_gene_map['Species'] == common_name_2, 'Real NCBI Gene'].to_list()))
    all_combo_list_current_genes = itertools.product(list_of_species_1_genes_in_group,list_of_species_2_genes_in_group)
    all_combo_list_current_genes = list(map(list,all_combo_list_current_genes))
    current_list_of_pairs = pd.DataFrame(all_combo_list_current_genes,columns = [common_name_1,common_name_2])
    current_list_of_pairs['Group Number'] = group_number
    all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.append(current_list_of_pairs)



all_pairs_to_evaluate_for_functional_conservation['Species 1 Score'] = np.nan
all_pairs_to_evaluate_for_functional_conservation['Species 2 Score'] = np.nan

## Trim cococonets to match


trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(cross_species_n_m_genes[common_name_1].to_list())]
trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(cross_species_n_m_genes[common_name_1].to_list())]
double_species_1_trimmed_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.columns.intersection(cross_species_map_one_to_one[common_name_1].to_list())]
double_species_1_trimmed_cococonet = double_species_1_trimmed_cococonet.replace(1,0)

trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(cross_species_n_m_genes[common_name_2].to_list())]
trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(cross_species_n_m_genes[common_name_2].to_list())]
double_species_2_trimmed_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.columns.intersection(cross_species_map_one_to_one[common_name_2].to_list())]
double_species_2_trimmed_cococonet = double_species_2_trimmed_cococonet.replace(1,0)



100%|██████████| 17785/17785 [02:23<00:00, 124.13it/s]


In [18]:
all_pairs_to_evaluate_for_functional_conservation

,mustard,arabidopsis,Group Number,Species 1 Score,Species 2 Score
0,LOC103859172,AT3G66656,OG0000487,NaN,NaN
1,LOC103859172,AT5G46320,OG0000487,NaN,NaN
2,LOC103859172,AT1G47760,OG0000487,NaN,NaN
3,LOC103859172,AT2G34440,OG0000487,NaN,NaN
4,LOC103865429,AT3G66656,OG0000487,NaN,NaN
...,...,...,...,...,...
0,LOC103856472,AT5G20910,OG0015144,NaN,NaN
0,LOC103868605,AT3G20540,OG0004539,NaN,NaN
1,LOC103868605,AT1G50840,OG0004539,NaN,NaN
2,LOC103869311,AT3G20540,OG0004539,NaN,NaN


In [19]:
double_species_1_trimmed_cococonet

,LOC103850518,LOC103860413,LOC103847160,LOC103850678,LOC103851253,LOC103851766,LOC103852362,LOC103856150,LOC103856741,LOC103859480,...,LOC103847450,LOC103847464,LOC103847466,LOC103847470,LOC103847473,LOC103847493,LOC103847505,LOC103847512,LOC103864791,LOC103848959
LOC103854507,0.829485,0.590667,0.760374,0.534088,0.908733,0.245048,0.569611,0.677289,0.717387,0.244821,...,0.877954,0.262383,0.622524,0.234584,0.079375,0.858260,0.674438,0.151878,0.352540,0.354966
LOC103827501,0.931780,0.731979,0.640242,0.822627,0.902736,0.458943,0.741064,0.980932,0.877465,0.741412,...,0.835667,0.599570,0.850798,0.156940,0.178878,0.930654,0.757500,0.851648,0.758776,0.597304
EXPA20,0.752532,0.595372,0.139228,0.740277,0.522279,0.186151,0.797820,0.806078,0.504094,0.668475,...,0.806783,0.140090,0.260191,0.052084,0.227455,0.767650,0.660144,0.946525,0.680222,0.591676
LOC103845929,0.432679,0.204430,0.496783,0.046104,0.570344,0.032486,0.103200,0.096440,0.081310,0.169514,...,0.190113,0.682316,0.193200,0.549403,0.064386,0.456364,0.306900,0.210414,0.177482,0.234712
LOC103850518,0.000000,0.820357,0.701516,0.932244,0.934298,0.657586,0.740142,0.972972,0.864745,0.376539,...,0.889918,0.480255,0.731211,0.390970,0.606507,0.910251,0.774208,0.580770,0.554973,0.792444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC103843841,0.988892,0.939014,0.521522,0.472783,0.951440,0.195527,0.549461,0.793946,0.615371,0.448220,...,0.957959,0.938235,0.458693,0.211787,0.289413,0.978151,0.879120,0.506086,0.845844,0.318708
LOC103848959,0.792444,0.432443,0.502075,0.989658,0.479485,0.920715,0.602548,0.826686,0.688596,0.234333,...,0.676994,0.146916,0.666038,0.460900,0.705241,0.365690,0.292377,0.318285,0.228600,0.000000
LOC103848960,0.505390,0.083016,0.181640,0.699324,0.252210,0.582906,0.240321,0.306381,0.405881,0.217869,...,0.358613,0.159003,0.130350,0.079180,0.519673,0.362942,0.233040,0.492857,0.040106,0.673717
LOC103848961,0.681507,0.129118,0.266414,0.663767,0.313095,0.646175,0.288104,0.525602,0.433178,0.168031,...,0.349980,0.356488,0.285529,0.101062,0.499442,0.426690,0.330118,0.492686,0.029439,0.695295


In [20]:
sum(all_pairs_to_evaluate_for_functional_conservation[common_name_1].isin(double_species_1_trimmed_cococonet.index.to_list()))

34945

In [21]:
all_pairs_to_evaluate_for_functional_conservation[common_name_2].isin(double_species_2_trimmed_cococonet.index)

0     True
1    False
2    False
3     True
4     True
     ...  
0     True
0     True
1     True
2     True
3     True
Name: arabidopsis, Length: 109541, dtype: bool

In [23]:
all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[common_name_1].isin(double_species_1_trimmed_cococonet.index)]
all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[common_name_2].isin(double_species_2_trimmed_cococonet.index)]


In [24]:
all_pairs_to_evaluate_for_functional_conservation

,mustard,arabidopsis,Group Number,Species 1 Score,Species 2 Score
0,LOC103860563,AT5G61010,OG0014200,NaN,NaN
0,LOC103833610,AT1G34000,OG0008009,NaN,NaN
1,LOC103840036,AT1G34000,OG0008009,NaN,NaN
0,LOC103848038,AT3G43800,OG0013247,NaN,NaN
0,LOC103873214,AT3G47700,OG0013170,NaN,NaN
...,...,...,...,...,...
0,LOC103856472,AT5G20910,OG0015144,NaN,NaN
0,LOC103868605,AT3G20540,OG0004539,NaN,NaN
1,LOC103868605,AT1G50840,OG0004539,NaN,NaN
2,LOC103869311,AT3G20540,OG0004539,NaN,NaN


In [ ]:


## Rank
species_1_cococonet_ranked = trimmed_species_1_cococonet.rank()
species_2_cococonet_ranked = trimmed_species_2_cococonet.rank()

#Do top 10 Genes
top_10_species_1_genes = np.array(
    [double_species_1_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_1_trimmed_cococonet.T]
)  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
top_10_species_1_genes_dataframe = pd.DataFrame(
    data=top_10_species_1_genes,
    index=double_species_1_trimmed_cococonet.index,
    columns=[
        "One",
        "Two",
        "Three",
        "Four",
        "Five",
        "Six",
        "Seven",
        "Eight",
        "Nine",
        "Ten",
    ],
)

#Convert 
top_10_species_1_genes_as_species_2 = top_10_species_1_genes_dataframe.replace(to_replace=dictionary_mapper_one_to_two)

# Get genes for checking 
have_species_1_pairs = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[common_name_1].isin(top_10_species_1_genes_as_species_2.index)]
trimmed_all_gene_pairs_for_fc = have_species_1_pairs.loc[have_species_1_pairs[common_name_2].isin(trimmed_species_2_cococonet.index)]
trimmed_all_gene_pairs_for_fc = trimmed_all_gene_pairs_for_fc.reset_index(drop = True)

# Get values in species 2 
for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
    current_species_1_gene = two_genes[1][common_name_1]
    current_species_2_gene = two_genes[1][common_name_2]
    finger_print_genes = top_10_species_1_genes_as_species_2.loc[current_species_1_gene].to_list()
    gene_ranks_in_species_2 = species_2_cococonet_ranked.loc[species_2_cococonet_ranked.index.isin(finger_print_genes), current_species_2_gene]
    avg_rank_in_species_2 = gene_ranks_in_species_2.mean()
    index_from_pairs = two_genes[0]
    trimmed_all_gene_pairs_for_fc.at[index_from_pairs, 'Species 1 Score'] = avg_rank_in_species_2

#Repeat for Species 2 

top_10_species_2_genes = np.array(
    [double_species_2_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_2_trimmed_cococonet.T]
)  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
top_10_species_2_genes_dataframe = pd.DataFrame(
    data=top_10_species_2_genes,
    index=double_species_2_trimmed_cococonet.index,
    columns=[
        "One",
        "Two",
        "Three",
        "Four",
        "Five",
        "Six",
        "Seven",
        "Eight",
        "Nine",
        "Ten",
    ],
)


#convert 
top_10_species_2_genes_as_species_1 = top_10_species_2_genes_dataframe.replace(to_replace=dictionary_mapper_dos_to_uno)


# Get values in species 1 
for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
    current_species_1_gene = two_genes[1][common_name_1]
    current_species_2_gene = two_genes[1][common_name_2]
    finger_print_genes = top_10_species_2_genes_as_species_1.loc[current_species_2_gene].to_list()
    gene_ranks_in_species_1 = species_1_cococonet_ranked.loc[species_1_cococonet_ranked.index.isin(finger_print_genes), current_species_1_gene]
    avg_rank_in_species_1 = gene_ranks_in_species_1.mean()
    index_from_pairs = two_genes[0]
    trimmed_all_gene_pairs_for_fc.loc[index_from_pairs, 'Species 2 Score'] = avg_rank_in_species_1

#Caluclate Divisors 
Number_of_species_1_genes = len(top_10_species_1_genes_as_species_2)
Number_of_species_2_genes = len(top_10_species_2_genes_as_species_1)


species_1_score_divisor = Number_of_species_2_genes - 4.5
species_2_score_divisor = Number_of_species_1_genes-4.5

#Divide and Average 
trimmed_all_gene_pairs_for_fc['Species 1 Score'] = trimmed_all_gene_pairs_for_fc['Species 1 Score']/species_1_score_divisor
trimmed_all_gene_pairs_for_fc['Species 2 Score'] = trimmed_all_gene_pairs_for_fc['Species 2 Score']/species_2_score_divisor
trimmed_all_gene_pairs_for_fc['Total Score'] = trimmed_all_gene_pairs_for_fc[['Species 1 Score','Species 2 Score']].mean(axis = 1)

In [98]:
all_pairs_to_evaluate_for_functional_conservation

,mustard,arabidopsis,Group Number,Species 1 Score,Species 2 Score


In [79]:
all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [common_name_1,common_name_2,'Group Number'])
for group_number in tqdm(list(set(outside_orthogroups['Orthogroup'].to_list()))):
    current_gene_map = outside_orthogroups.loc[outside_orthogroups['Orthogroup'] == group_number]
    list_of_species_1_genes_in_group = list(set(current_gene_map.loc[current_gene_map['Species'] == common_name_1, 'Real NCBI Gene'].to_list()))
    list_of_species_2_genes_in_group = list(set(current_gene_map.loc[current_gene_map['Species'] == common_name_2, 'Real NCBI Gene'].to_list()))
    all_combo_list_current_genes = itertools.product(list_of_species_1_genes_in_group,list_of_species_2_genes_in_group)
    all_combo_list_current_genes = list(map(list,all_combo_list_current_genes))
    current_list_of_pairs = pd.DataFrame(all_combo_list_current_genes,columns = [common_name_1,common_name_2])
    current_list_of_pairs['Group Number'] = group_number
    all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.append(current_list_of_pairs)
    break

  0%|          | 0/17785 [00:00<?, ?it/s]


In [93]:
outside_orthogroups

,Orthogroup,Protein,Species,Real NCBI Gene
0,OG0000000,XP_009123338.2,rapeseed,LOC103848115
1,OG0000000,XP_009123420.1,rapeseed,LOC103848216
2,OG0000000,XP_009123559.2,rapeseed,LOC103848395
3,OG0000000,XP_009124167.1,rapeseed,LOC103849105
4,OG0000000,XP_009135775.2,rapeseed,LOC103859933
...,...,...,...,...
204014,OG0025193,XP_033148476.1,rapeseed,LOC103868320
204016,OG0025194,XP_033148550.1,rapeseed,LOC103874885
204018,OG0025195,XP_033148563.1,rapeseed,LOC117134375
204020,OG0025196,XP_033148644.1,rapeseed,LOC103869108


In [83]:
common_name_1

'mustard'

In [87]:
current_gene_map

,Orthogroup,Protein,Species,Real NCBI Gene
26202,OG0000248,AT1G03400,arabidopsis,AT1G03400
26203,OG0000248,AT1G03410,arabidopsis,AT1G03410
26204,OG0000248,AT1G06620,arabidopsis,AT1G06620
26205,OG0000248,AT1G06640,arabidopsis,AT1G06640
26206,OG0000248,AT1G06645,arabidopsis,AT1G06645
26207,OG0000248,AT1G06650,arabidopsis,AT1G06650
26208,OG0000248,AT2G25450,arabidopsis,AT2G25450
26209,OG0000248,AT2G30830,arabidopsis,AT2G30830
26210,OG0000248,AT2G30840,arabidopsis,AT2G30840
26211,OG0000248,AT3G61400,arabidopsis,AT3G61400


In [82]:
current_gene_map.loc[current_gene_map['Species'] == common_name_1]

,Orthogroup,Protein,Species,Real NCBI Gene


In [80]:
list_of_species_1_genes_in_group

[]

In [76]:
all_pairs_to_evaluate_for_functional_conservation

,mustard,arabidopsis,Group Number


In [25]:
brapa_thaliana_scores = Calculate_Score_list_for_thresholding_and_use_alternative_list_for_ogs(Species_1 = 'mustard',Species_2 = 'arabidopsis',outside_orthogroups = just_brapa_and_athaliana)
brapa_thaliana_scores

100%|██████████| 17785/17785 [02:51<00:00, 103.79it/s]


,mustard,arabidopsis,Group Number,Species 1 Score,Species 2 Score,Total Score
0,LOC103860563,AT5G61010,OG0014200,0.910743,0.961194,0.935968
1,LOC103833610,AT1G34000,OG0008009,0.988080,0.993365,0.990722
2,LOC103840036,AT1G34000,OG0008009,0.987229,0.991107,0.989168
3,LOC103848038,AT3G43800,OG0013247,0.833103,0.742984,0.788043
4,LOC103873214,AT3G47700,OG0013170,0.942415,0.962768,0.952592
...,...,...,...,...,...,...
33111,LOC103856472,AT5G20910,OG0015144,0.965722,0.863680,0.914701
33112,LOC103868605,AT3G20540,OG0004539,0.983632,0.970260,0.976946
33113,LOC103868605,AT1G50840,OG0004539,0.993202,0.986666,0.989934
33114,LOC103869311,AT3G20540,OG0004539,0.982882,0.963808,0.973345


In [33]:
brapa_thaliana_scores.mean()

Species 1 Score    0.846737
Species 2 Score    0.817102
Total Score        0.829443
dtype: float64

In [28]:
brapa_thaliana_scores.to_csv('/data/passala/Collaborator_Data/Nick_cross_species_evolution/Updated_orthogroups/brapa_thaliana_scores.csv',index = False)

In [26]:
bnapus_thaliana_scores = Calculate_Score_list_for_thresholding_and_use_alternative_list_for_ogs(Species_1 = 'rapeseed',Species_2 = 'arabidopsis',outside_orthogroups = just_bnapus_and_athaliana)
bnapus_thaliana_scores

100%|██████████| 18868/18868 [04:08<00:00, 75.99it/s]


,rapeseed,arabidopsis,Group Number,Species 1 Score,Species 2 Score,Total Score
0,LOC106410754,AT5G61010,OG0014200,0.576142,0.419416,0.497779
1,LOC106439950,AT5G61010,OG0014200,0.511327,0.345381,0.428354
2,LOC111197979,AT1G34000,OG0008009,0.699319,0.017478,0.358399
3,LOC106402491,AT3G43800,OG0013247,0.644487,0.462133,0.553310
4,LOC106347702,AT3G43800,OG0013247,0.530657,0.506352,0.518505
...,...,...,...,...,...,...
68782,LOC106451652,AT1G50840,OG0004539,0.657870,0.187728,0.422799
68783,LOC106357489,AT3G20540,OG0004539,0.524145,0.241825,0.382985
68784,LOC106357489,AT1G50840,OG0004539,0.573469,0.314119,0.443794
68785,LOC106385435,AT3G20540,OG0004539,0.598842,0.088909,0.343875


In [32]:
bnapus_thaliana_scores.mean()

Species 1 Score    0.606372
Species 2 Score    0.392254
Total Score        0.499789
dtype: float64

In [29]:
bnapus_thaliana_scores.to_csv('/data/passala/Collaborator_Data/Nick_cross_species_evolution/Updated_orthogroups/bnapus_thaliana_scores.csv',index = False)

In [27]:
bnapus_brapa_scores = Calculate_Score_list_for_thresholding_and_use_alternative_list_for_ogs(Species_1 = 'rapeseed',Species_2 = 'mustard',outside_orthogroups = just_bnapus_and_brapa)
bnapus_brapa_scores

100%|██████████| 18331/18331 [06:04<00:00, 50.27it/s]


,rapeseed,mustard,Group Number,Species 1 Score,Species 2 Score,Total Score
0,LOC106410754,LOC103860563,OG0014200,0.494843,0.395427,0.445135
1,LOC106439950,LOC103860563,OG0014200,0.521238,0.552425,0.536832
2,LOC111197979,LOC103833610,OG0008009,0.416784,0.039760,0.228272
3,LOC111197979,LOC103840036,OG0008009,0.362701,0.161918,0.262310
4,LOC106402491,LOC103848038,OG0013247,0.475936,0.499971,0.487954
...,...,...,...,...,...,...
76227,LOC106451652,LOC103869311,OG0004539,0.512997,0.199081,0.356039
76228,LOC106357489,LOC103868605,OG0004539,0.520415,0.243333,0.381874
76229,LOC106357489,LOC103869311,OG0004539,0.497016,0.147769,0.322393
76230,LOC106385435,LOC103868605,OG0004539,0.439982,0.069821,0.254901


In [31]:
bnapus_brapa_scores.mean()

Species 1 Score    0.518967
Species 2 Score    0.340501
Total Score        0.429719
dtype: float64

In [30]:
bnapus_brapa_scores.to_csv('/data/passala/Collaborator_Data/Nick_cross_species_evolution/Updated_orthogroups/bnapus_brapa_scores.csv',index = False)

In [41]:
common_name_1 = Name_resolver.species_name_resolver('rapeseed','common')
common_name_2 = Name_resolver.species_name_resolver('arabidopsis','common')
if common_name_1 == 'rice':
    common_name_1 = 'rice_jp'
if common_name_2 == 'rice':
    common_name_2 ='rice_jp'
file_list = os.listdir('/data/passala/OrthoDB_data/V_11_pairwise_maps_fixed_problem_species')
spec_1_filter = [i for i in file_list if common_name_1 in i]
spec_2_filter = [i for i in spec_1_filter if common_name_2 in i]
current_location = '/data/passala/OrthoDB_data/V_11_pairwise_maps_fixed_problem_species/' + spec_2_filter[0] #get string for map

IndexError: list index out of range

In [47]:
common_name_1

'rapeseed'

In [46]:
file_list

['rice_jp_to_apple_ortholog_NM.csv',
 'rice_jp_to_sorghum_ortholog_NM.csv',
 'rice_jp_to_maize_ortholog_NM.csv',
 'rice_jp_to_grape_ortholog_NM.csv',
 'rice_jp_to_mustard_ortholog_NM.csv',
 'rice_jp_to_soybean_ortholog_NM.csv',
 'rice_jp_to_medicago_ortholog_NM.csv',
 'rice_jp_to_tobacco_ortholog_NM.csv',
 'rice_jp_to_potato_ortholog_NM.csv',
 'rice_jp_to_tomato_ortholog_NM.csv',
 'rice_jp_to_arabidopsis_ortholog_NM.csv',
 'rice_jp_to_peanut_ortholog_NM.csv',
 'rice_jp_to_wheat_ortholog_NM.csv',
 'rice_jp_to_cucumber_ortholog_NM.csv',
 'rice_jp_to_brome_ortholog_NM.csv',
 'apple_to_sorghum_ortholog_NM.csv',
 'apple_to_maize_ortholog_NM.csv',
 'apple_to_grape_ortholog_NM.csv',
 'apple_to_mustard_ortholog_NM.csv',
 'apple_to_soybean_ortholog_NM.csv',
 'apple_to_medicago_ortholog_NM.csv',
 'apple_to_tobacco_ortholog_NM.csv',
 'apple_to_potato_ortholog_NM.csv',
 'apple_to_tomato_ortholog_NM.csv',
 'apple_to_arabidopsis_ortholog_NM.csv',
 'apple_to_peanut_ortholog_NM.csv',
 'apple_to_wheat_